In [2]:
# encoding:utf-8
# FileName: send_email
# Author:   wzg
# email:    1010490079@qq.com
# Date:     2019/12/1 下午 06:59
# Description: 邮件发送--带附件


import smtplib
from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime
import pandas as pd

def get_token(date):
    df = pd.read_csv('./logs/' + date + '.txt', delimiter='\t')
    df['question_token_count'] = df['question'].apply(lambda x:len(x))
    df['answer_token_count'] = df['answer'].apply(lambda x:len(x))
    # 计算总token数
    q_tokens = df['question_token_count'].sum()
    a_tokens = df['answer_token_count'].sum()
    return q_tokens, a_tokens, len(df)
if __name__ == '__main__':
    # 发件人邮箱
    username = "2464367707@qq.com"
    # QQ邮箱授权码
    password = "nqtdpthnjnbeecfe"
    # 可设置多个收件人邮箱
    receivers = ['3408853453@qq.com']
    # 设置抄送人信息，可多个，逗号分隔
    cc = ['2464367707@qq.com']

    now = datetime.now()
    timestamp = now.strftime('%Y-%m-%d')
    q_tokens, a_tokens,times = get_token(timestamp)
    # 设置标题
    subject = timestamp + " chatgpt网页端：{}Token({}次)({}元)".format(q_tokens+a_tokens, times, round((q_tokens+a_tokens)/1000*0.002*6,4))
    print(subject)
    # 设置内容
    
    content = "chat gpt 网页端{}统计情况\n问题请求tokens：{}\n回答请求tokens：{}".format(timestamp,q_tokens,a_tokens)
    print(content)
    # 创建MIMEMultipart对象，并封装相应的数据
    message = MIMEMultipart()
    # 封装标题
    message['Subject'] = Header(subject, 'gbk')
    # 封装发件人标识
    message['From'] = "chatgpt网页端Token统计"
    # 封装收件人和抄送人
    message['to'] = Header(",".join(receivers))
    message['Cc'] = Header(",".join(cc))
    # 生成邮件正文
    send_text = MIMEText(content, "plain", "utf-8")
    # # 封装邮件正文
    message.attach(send_text)
    # message.attach(MIMEText(content, 'plain'))

    # 读取csv文件作为附件
    send_file_path = './logs/' + timestamp + '.txt'
    # 发送附件
    add_file = MIMEText(open(send_file_path, 'rb').read(), 'base64', 'gbk')
    add_file['Content-Type'] = 'application/octet-stream'
    # 设置文件名称
    add_file.add_header('Content-Disposition', 'attachment', filename="{}".format(send_file_path.split("\\")[-1]))
    message.attach(add_file)

    try:
        # 设置smtp的相关参数
        smtp_server = 'smtp.qq.com'
        smtp_port = 25
        smtp = smtplib.SMTP(smtp_server, smtp_port)
        # 登录
        smtp.login(username, password)
        # 发送
        smtp.sendmail(username, receivers + cc, message.as_string())
        smtp.quit()
        print("邮件发送成功")
    except smtplib.SMTPException as e:
        print("Error: 无法发送邮件" + e.strerror)

2023-03-07 chatgpt网页端：833Token(4次)(0.01元)
chat gpt 网页端2023-03-07统计情况
问题请求tokens：66
回答请求tokens：767
邮件发送成功


In [20]:
s = '一个python pat cat'
pattern = re.compile(r'([\u4e00-\u9fa5]+|\w+|\|\|)')
# 使用正则表达式分隔字符串，并计算每个token的长度
token_lengths = [len(token) for token in pattern.findall(s)]
print(token_lengths)
pattern.findall(s)

[2, 6, 3, 3]


['一个', 'python', 'pat', 'cat']

In [16]:
import pandas as pd
import re
# 读取表格数据
df = pd.read_csv('./logs/2023-03-06.txt', delimiter='\t')

# 定义count_tokens函数，计算一个字符串的token数
def count_tokens(s):
    # 定义正则表达式，匹配中文字符和英文单词
    pattern = re.compile(r'([\u4e00-\u9fa5]+|\w+|\|\|)')
    # 使用正则表达式分隔字符串，并计算每个token的长度
    token_lengths = [len(token) for token in pattern.findall(s)]
    # 返回总长度
    return sum(token_lengths)

# 计算question列的token数
df['question_token_count'] = df['question'].apply(count_tokens)
print(df['question_token_count'])
# 计算answer列的token数
df['answer_token_count'] = df['answer'].apply(count_tokens)
print(df['answer_token_count'])
# 计算总token数
total_token_count = df['question_token_count'].sum() + df['answer_token_count'].sum()

# 打印结果
# print(df[['question', 'question_token_count', 'answer', 'answer_token_count']])
print('\nTotal token count:', total_token_count)


    

0     5
1     6
2     5
3    47
Name: question_token_count, dtype: int64
0     21
1    121
2     57
3    433
Name: answer_token_count, dtype: int64

Total token count: 695
